# Setup prompts and interact with IBM BAM

In this notebook, we will interact with IBM models and setup input prompts to get started with the api docs generator.

In [1]:
import os
import json
import re
import pandas as pd
from dotenv import load_dotenv
from genai.credentials import Credentials
from genai.model import Model
from genai.schemas import GenerateParams

/opt/app-root/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup BAM API

In [2]:
# make sure you have a .env file in the root folder with genaikey and genaiapi
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)

In [3]:
# creds object
creds = Credentials(api_key=api_key, api_endpoint=api_endpoint)

# Instantiate parameters for text generation
params = GenerateParams(
    decoding_method="sample",
    max_new_tokens=1024,
    temperature=0.7
)

model = "meta-llama/llama-2-7b"
MAX_TOKEN_LENGTH = 4096
# Instantiate a model proxy object to send your requests
chosen_model = Model(model, params=params, credentials=creds)

# Instantiate a paramater dictionary object
params_dict = vars(params)

## Data

In [4]:
import json

In [5]:
# load markdown data
dataset_path = "../../data/raw/markdown_data.json"
with open(dataset_path, 'r') as f:
		markdown = json.load(f)

# load nested data
dataset_path = "../../data/raw/nested_data.json"
with open(dataset_path, 'r') as f:
		data = json.load(f)

In [6]:
markdown.keys()

dict_keys(['errors', 'oidc', 'sign', 'transparency', 'verify'])

In [7]:
data.keys()

dict_keys(['errors', 'oidc', 'sign', 'transparency', 'verify_models', 'verify_policy', 'verify_verifier'])

In [8]:
code_snippet = data["oidc"]["code"][0]

In [9]:
code_snippet.keys()

dict_keys(['oidc.py'])

In [10]:
code_snippet = code_snippet["oidc.py"]

In [11]:
print(code_snippet)

# Copyright 2022 The Sigstore Authors
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
API for retrieving OIDC tokens.
"""

from __future__ import annotations

import logging
import sys
import time
import urllib.parse
import webbrowser
from datetime import datetime, timezone
from typing import NoReturn, Optional, cast

import id
import jwt
import requests
from pydantic import BaseModel, StrictStr

from sigstore.errors import Error, NetworkError

DEFAULT_OAUTH_ISSUER_URL = "https://oauth2.sigs

In [12]:
len(code_snippet)

15939

Since the length of the code is big, let's try the prompts on a smaller code piece.

In [13]:
def split_code_into_pieces(code):
    # Use regular expressions to split based on patterns (e.g., functions, classes)
    pattern = r'\b(?:def|class)\s+[a-zA-Z_][a-zA-Z0-9_]*\s*\([^)]*\)\s*:'
    code_pieces = re.split(pattern, code)
    
    # Include the matched patterns as separators between code pieces
    separators = re.findall(pattern, code)
    
    # Combine the code pieces and separators into a list
    code_sections = [piece.strip() for piece in code_pieces if piece.strip()]
    
    # Combine code pieces with separators in between
    result_code = []
    for i, section in enumerate(code_sections):
        result_code.append(section)
        if i < len(separators):
            result_code.append(separators[i])

    return result_code

In [14]:
code_pieces = split_code_into_pieces(code_snippet)

In [15]:
print(code_pieces[6])

"""
    Raised on any communication or format error with an OIDC issuer.
    """

    pass


class Issuer:
    """
    Represents an OIDC issuer (IdP).
    """

    def __init__(self, base_url: str) -> None:
        """
        Create a new `Issuer` from the given base URL.

        This URL is used to locate an OpenID Connect configuration file,
        which is then used to bootstrap the issuer's state (such
        as authorization and token endpoints).
        """
        oidc_config_url = urllib.parse.urljoin(
            f"{base_url}/", ".well-known/openid-configuration"
        )

        try:
            resp: requests.Response = requests.get(oidc_config_url, timeout=30)
        except (requests.ConnectionError, requests.Timeout) as exc:
            raise NetworkError from exc

        try:
            resp.raise_for_status()
        except requests.HTTPError as http_error:
            raise IssuerError from http_error

        try:
            # We don't generally expect this 

### Generate prompts

In [16]:
def generate_text(chosen_model, prompt):
    response = chosen_model.generate([prompt_1])
    generated_text = response[0].generated_text
    return generated_text

#### Prompt 1

In [17]:
# Example prompt 1
prompt_1 = """
Generate documentation for each function in the given 
Code snippet:{code_snippet}

Function or class name

""".format(code_snippet=code_pieces[6])

In [18]:
print(generate_text(chosen_model, prompt_1))

"""
    def id_token(  # nosec: B107
        self,
        force_oob: bool = False,
    ) -> IdentityToken:
        """
        Retrieves and returns an `IdentityToken` from the current `Issuer`, via OAuth.

        This function blocks on user interaction.

        The `force_oob` flag controls the kind of flow performed. When `False` (the default),
        this function attempts to open the user's web browser before falling back to
        an out-of-band flow. When `True`, the out-of-band flow is always used.
        """

        # This function and the components that it relies on are based off of:
        # https://github.com/psteniusubi/python-sample

        from sigstore._internal.oidc.oauth import _OAuthFlow

        code: str
        with _OAuthFlow(self) as server:
            # Launch web browser
            if not force_oob and webbrowser.open(server.base_uri):
                print("Waiting for browser interaction...", file=sys.stderr)
            else:
                ser

#### Prompt 2

In [19]:
# Example prompt 2
prompt_2 = """
Generate documentation for each function in the given code snippet.

1. Function name 
2. Function description 
3. Input arguments
4. Parameters

Code snippet:
{code_snippet}
""".format(code_snippet=code_pieces[6])

In [20]:
print(generate_text(chosen_model, prompt_2))

    def __init__(self, base_url: str) -> None:
        """
        Create a new `Issuer` from the given base URL.

        This URL is used to locate an OpenID Connect configuration file,
        which is then used to bootstrap the issuer's state (such
        as authorization and token endpoints).
        """
        oidc_config_url = urllib.parse.urljoin(
            f"{base_url}/", ".well-known/openid-configuration"
        )

        try:
            resp: requests.Response = requests.get(oidc_config_url, timeout=30)
        except (requests.ConnectionError, requests.Timeout) as exc:
            raise NetworkError from exc

        try:
            resp.raise_for_status()
        except requests.HTTPError as http_error:
            raise IssuerError from http_error

        try:
            # We don't generally expect this to fail (since the provider should
            # return a non-success HTTP code which we catch above), but we
            # check just in case we have a misbehav

#### Expected Output

In [21]:
expected = data['oidc']['markdown']
print(expected)

[ sigstore](../sigstore.html)

## API Documentation

  * DEFAULT_OAUTH_ISSUER_URL
  * STAGING_OAUTH_ISSUER_URL
  * DEFAULT_AUDIENCE
  * ExpiredIdentity
  * IdentityToken
    * IdentityToken
    * in_validity_period
    * identity
    * issuer
    * expected_certificate_subject
  * IssuerError
  * Issuer
    * Issuer
    * production
    * staging
    * identity_token
  * IdentityError
    * raise_from_id
    * diagnostics
  * detect_credential

[ built with pdoc ](https://pdoc.dev "pdoc: Python API documentation
generator")

#  [sigstore](./../sigstore.html).oidc

API for retrieving OIDC tokens.

View Source

    
    
      1# Copyright 2022 The Sigstore Authors
      2#
      3# Licensed under the Apache License, Version 2.0 (the "License");
      4# you may not use this file except in compliance with the License.
      5# You may obtain a copy of the License at
      6#
      7#      http://www.apache.org/licenses/LICENSE-2.0
      8#
      9# Unless required by applicable law or ag